In [1]:
import csv
import sys, os
import pandas as pd
import numpy as np

The basic code outline is as follows:   
0. Import both *talks_6e.csv* and *metadata_all.csv*. Create output file *talks_6f.csv*
1. Sort each import by the author name
2. ~~For *metadata_all.csv*, create a new colum that breaks the slug up into a list of lower-cased words.~~ 
3. Create an index list for *metadata_all.csv*. The idea of this list is to remove rows from consideration as we match the slugs to rows in *talks_6e.csv*.
4. Loop over each row in *talks_6e.csv*:
  1. Create two new lists to use: 
    * The list of names, all lower cased
    * The list of words in the title, all lowered case
  2. Iterate over each available row in *metadata_all.csv*, matching the author lists to the slug lists. 
    * If the author and slug lists intersect over at least 50% of the words in the author list, then compare the title and slug lists
    * If the title and slug lists match over 50% of the words in the title list, then match these rows and create a new row in the output that has the information from both *talks_6e.csv* and *metadata_all.csv*. Then move on to the next row of *metadata_all.csv*.
    * If no match is found, move on to the next row of *metadata_all.csv*.

In [29]:
# Step 0 - Import CSV files
read1 = csv.reader(open('../data/talks_6e.csv'))
headers = next(read1)

read2 = csv.reader(open('../../tedmetadata/metadata_all.csv'))
head2 = next(read2)

# Create headers for the output file, such that:
#    * 'date' --> 'published'
#    * 'numDate' --> numPub'

ind = headers.index('date')
headers[ind] = 'published'

ind = headers.index('numDate')
headers[ind] = 'numPub'

# 2) Add headers from the *metadata_all.csv* file
headers = [head2[0]] + headers[1:] + head2[1:] + ['numFilmed']

# Set index names:
#aund = headers.index('author')
#tnd = headers.index('title')



In [30]:
#test = next(read1)
#print(type(test))

In [31]:
#print(headers)
#print(head2)

In [32]:
#len(talks6)

In [33]:
#talks6.iloc[0].name

In [34]:
#talks6.loc[talks6.iloc[0].name].tolist()[1:]

In [35]:
#row_test = talks6.loc[talks6.iloc[0].name]
#row_test["author"].lower().split()

Here's the thinking.... maybe using pandas we can do all the things in place? Create a few fake lists that we add values to as we go. Then create a new pandas dataframe that has all the pieces. 

It seems to be easy to add a column to a pandas dataframe, but you have to have a full column. So basically, we pull each talk, add the info we need to new lists at the index that things will appear (using a hack of iloc+loc) and then add the new columns after the loop....

In [36]:
# Step 1 - Sort each import: one by slug, the other by author
read1pd = pd.read_csv('../data/talks_6e.csv')
talks6 = read1pd.sort_values("author", kind="mergesort")

# Import *metadata_all.csv* as pandas data frame
meta_raw = pd.read_csv('../../tedmetadata/metadata_all.csv')
meta = meta_raw.sort_values("slug", kind="mergesort")

In [37]:
#row2 = meta.iloc[2]
#slist = row2['slug']
#type(slist)

In [38]:
# Step 2 - For *metadata_all.csv*, create a dataframe with a new column where there is a list of words
# jk, we're just going to do this in the loop

In [39]:
# Step 3 - Create an index list for *metadata_all.csv*
mnds = list(range(len(meta)))

In [40]:
# Step 4 - Loop over the rows in *talks_6e.csv*
with open('../data/talks_7.csv', 'w') as csvfile:
    # Make the output file
    outfile = csv.writer(csvfile)
    outfile.writerow(headers)
    
    matchless_inds = []
    
    # Loop over each row in *talks_6e.csv*
    for inds in range(len(talks6)):
        talk_row = talks6.loc[talks6.iloc[inds].name]
        aulist = talk_row["author"].lower().split()
        tlist = talk_row["title"].lower().split()
        
        good_inds = []
        for m in mnds:
            # Grab row associated to index M in *metadata_all.csv*
            row2 = meta.iloc[m]
            slist = row2['slug'].lower().split('_')
            
            # Compare author and slug lists:
            if len(set(aulist) & set(slist)) >= 0.5*len(aulist):
                # If comparison goes well, then compare title and slug lists:
                if len(set(tlist) & set(slist)) > 0.5*len(tlist):
                    good_inds.append(m)
                    
        if len(good_inds) == 1:
            gi = good_inds[0]
            row_gi = meta.iloc[gi]
            # If we find a match, then we need to add the output row: 
            add_row = [row_gi['slug']] + talk_row.tolist()[1:]
            add_row.append(row_gi['view_count'])
            add_row.append(row_gi['event'])
            add_row.append(row_gi['filmed'])
            numFilmed = ''.join(row_gi["filmed"].split('-'))
            add_row.append(numFilmed)
                
            outfile.writerow(add_row)
                
            mnds.remove(gi)

        else:
            matchless_inds.append(inds)

In [41]:
print(len(mnds))
print(len(meta))
print(len(talks6))

1083
2628
2067


In [42]:
talks_test = pd.read_csv('../data/talks_7.csv')
talks_test

,slug,citation,author,gender,title,published,length,text,occupation,numPub,ValenceScore,view_count,event,filmed,numFilmed
0,aj_jacobs_how_healthy_living_nearly_killed_me,AJ Jacobs 2012,AJ Jacobs,male,How healthy living nearly killed me,Jan 2012,509,I've spent the last decade subjecting myself t...,Author,201201,6.099078,1511571,TEDMED 2011,2011-10-28,20111028
1,aj_jacobs_the_world_s_largest_family_reunion_w...,AJ Jacobs 2014,AJ Jacobs,male,The world's largest family reunion … we're all...,Jun 2014,577,Six months ago I got an email from a man in I...,Author,201406,6.267381,1123811,TEDActive 2014,2014-03-20,20140320
2,a_j_jacobs_year_of_living_biblically,AJ Jacobs 2008,AJ Jacobs,male,My year of living biblically,Jul 2008,1023,I thought I'd tell you a little about what I l...,Author,200807,6.075994,2419084,EG 2007,2007-12-02,20071202
3,aaron_o_connell_making_sense_of_a_visible_quan...,Aaron O'Connell 2011,Aaron O'Connell,male,Making sense of a visible quantum object,Jun 2011,459,This is a representation of your brain and yo...,Physicist,201106,6.058835,1290118,TED2011,2011-03-02,20110302
4,abe_davis_new_video_technology_that_reveals_an...,Abe Davis 2015,Abe Davis,male,New video technology that reveals an object's ...,May 2015,1066,Most of us think of motion as a very visual th...,Computer scientist,201505,6.269410,1405188,TED2015,2015-03-17,20150317
5,abha_dawesar_life_in_the_digital_now,Abha Dawesar 2013,Abha Dawesar,female,"Life in the ""digital now""",Oct 2013,708,I was in New York during Hurricane Sandy and ...,Novelist,201310,6.091329,1263264,TEDGlobal 2013,2013-06-14,20130614
6,abigail_washburn_building_us_china_relations_b...,Abigail Washburn 2012,Abigail Washburn,female,Building US-China relations ... by banjo,Apr 2012,373,If you had caught me straight out of college i...,Clawhammer banjo player,201204,6.322439,989838,TED2012,2012-02-27,20120227
7,abraham_verghese_a_doctor_s_touch,Abraham Verghese 2011,Abraham Verghese,male,A doctor's touch,Sep 2011,1090,A few months ago a year old woman came to ...,Physician and author,201109,5.877625,1432826,TEDGlobal 2011,2011-07-14,20110714
8,achenyo_idachaba_how_i_turned_a_deadly_plant_i...,Achenyo Idachaba 2016,Achenyo Idachaba,female,How I turned a deadly plant into a thriving bu...,Jan 2016,431,Welcome to Bayeku a riverine community in Iko...,Green entrepreneur,201601,6.207708,1743873,TEDWomen 2015,2015-05-27,20150527
9,adam_davidson_what_we_learned_from_teetering_o...,Adam Davidson 2012,Adam Davidson,male,What we learned from teetering on the fiscal c...,Dec 2012,1132,So a friend of mine who's a political scientis...,Journalist,201212,5.755019,787596,TEDSalon NY2012,2012-12-17,20121217


In [43]:
# Output files test

unmatchedMeta = meta.iloc[mnds]
unmatchedMeta.to_csv('../../tedmetadata/unmatched_meta.csv')
unmatchedTalks = talks6.iloc[matchless_inds]
unmatchedTalks.to_csv('../data/unmatched_talks6e.csv')

In [44]:
umMeta = pd.read_csv('../../tedmetadata/unmatched_meta.csv')
print(len(umMeta))
#umMeta

1083


In [45]:
umTalks = pd.read_csv('../data/unmatched_talks6e.csv')
print(len(umTalks))
#umTalks

522


In [27]:
# Next wave of matching. Noticing that many people give exactly one talk, we will use the unmatched files
# to make more rows in *talks_7.csv*

# We will follow nearly the same procedure as before EXCEPT that the unmatch files are already sorted. 

# Step 1 - Import the files & get the headers: 
read_talks = csv.reader(open('../data/talks_7.csv'))
headers = next(read_talks)

UMtalks6 = pd.read_csv('../data/unmatched_talks6e.csv')
UMmeta = pd.read_csv('../../tedmetadata/unmatched_meta.csv')

# Step 2 - DNE

# Step 3 - Create an index list for *unmatched_meta.csv*
mnds = list(range(len(UMmeta)))

# Step 4 - Loop over the rows in *talks_6e.csv*
with open('../data/talks_7add1.csv', 'w') as csvfile:
    # Make the output file
    outfile = csv.writer(csvfile)
    outfile.writerow(headers)
    
    matchless_inds = []
    
    # Loop over each row in *talks_6e.csv*
    for inds in range(len(UMtalks6)):
        talk_row = UMtalks6.iloc[inds]
        aulist = talk_row["author"].lower().split()
        tlist = talk_row["title"].lower().split()
        
        # Set MATCH flag
        match = False
        good_inds = []
        
        for m in mnds:
            # Grab row associated to index M in *unmatched_meta.csv*
            row2 = UMmeta.iloc[m]
            slist = row2['slug'].lower().split('_')
            
            # Compare author and slug lists:
            if len(set(aulist) & set(slist)) == len(aulist):
                # If comparison goes well, add this index to GOOD_INDS
                good_inds.append(m)
                #print("match!")
                #print(good_inds)
                
        #print("For talks row", str(inds), "there are ", str(len(good_inds)), "matches")
        if len(good_inds) == 1:
            # if we find exactly one match by author, then we can add the
            # the row to the output. 
            gi = good_inds[0]
            row_gi = UMmeta.iloc[gi]
            
            add_row = [row_gi['slug']] + talk_row.tolist()[:]
            add_row.append(row_gi['view_count'])
            add_row.append(row_gi['event'])
            add_row.append(row_gi['filmed'])
            numFilmed = ''.join(row_gi["filmed"].split('-'))
            add_row.append(numFilmed)
            
            print(talk_row.tolist()[1:])
                
            outfile.writerow(add_row)
                
            mnds.remove(gi)
        
        else :
            matchless_inds.append(inds)

unmatchedMeta2 = UMmeta.iloc[mnds]
unmatchedMeta2.to_csv('../../tedmetadata/unmatched_meta2.csv')
unmatchedTalks2 = UMtalks6.iloc[matchless_inds]
unmatchedTalks2.to_csv('../data/unmatched_talks6e2.csv')

[1828, 'Aaron Huey 2010', 'Aaron Huey', 'male', "America's native prisoners of war", 'Nov 2010', 911, "I'm here today to show my photographs of the Lakota. Many of you may have heard of the Lakota  or at least the larger group of tribes called the Sioux. The Lakota are one of many tribes that were moved off their land to prisoner of war camps now called reservations. The Pine Ridge Reservation  the subject of today's slide show  is located about    miles southeast of the Black Hills in South Dakota. It is sometimes referred to as Prisoner of War Camp Number      and it is where the Lakota now live. Now  if any of you have ever heard of AIM  the American Indian Movement  or of Russell Means  or Leonard Peltier  or of the stand off at Oglala  then you know that Pine Ridge is ground zero for Native issues in the U.S. So I've been asked to talk a little bit today about my relationship with the Lakota  and that's a very difficult one for me. Because  if you haven't noticed from my skin colo

[1251, 'Adam Grosser 2008', 'Adam Grosser', 'male', 'A mobile fridge for vaccines', 'Jun 2008', 201, "This is a work in process  based on some comments that were made at TED two years ago about the need for the storage of vaccine.  .  billion people don't have access to electricity  refrigeration or stored fuels. This is a problem. It impacts  the spread of disease  the storage of food and medicine and the quality of life. So here's the plan  inexpensive refrigeration that doesn't use electricity  propane  gas  kerosene or consumables. Time for some thermodynamics. And the story of the Intermittent Absorption Refrigerator. Adam Grosser  So    years ago  I had this thermo teacher who talked about absorption and refrigeration. It's one of those things that stuck in my head. It was a lot like the Stirling engine  it was cool  but you didn't know what to do with it. And it was invented in       by this guy Ferdinand Carre  but he couldn't actually build anything with it because of the tool

[1881, 'Al Seckel 2007', 'Al Seckel', 'male', 'Visual illusions that show how we (mis)think', 'Apr 2007', 778, "We're going to talk    my    a new lecture  just for TED    and I'm going show you some illusions that we've created for TED  and I'm going to try to relate this to happiness. What I was thinking about with happiness is  what gives happiness    or happiness  which I equate with joy in my particular area  and I think there's something very fundamental. And I was thinking about this. And it's in terms of both illusions and movies that we go see and jokes and magic shows is that there's something about these things where our expectations are violated in some sort of pleasing way. You go see a movie. And it has an unexpected twist    something that you didn't expect    and you find a joyful experience. You look at those sort of illusions in my book and it's not as what you'd expect. And there's something joyful about it. And it's the same thing with jokes and all these sorts of t

[197, 'Alan Russell 2007', 'Alan Russell', 'male', 'The potential of regenerative medicine', 'Jul 2007', 1110, "I'm going to talk to you today about hopefully converting fear into hope. When we go to the physician today    when we go to the doctor's office and we walk in  there are words that we just don't want to hear. There are words that we're truly afraid of. Diabetes  cancer  Parkinson's  Alzheimer's  heart failure  lung failure    things that we know are debilitating diseases  for which there's relatively little that can be done. And what I want to lay out for you today is a different way of thinking about how to treat debilitating disease  why it's important  why without it perhaps our health care system will melt down if you think it already hasn't  and where we are clinically today  and where we might go tomorrow  and what some of the hurdles are. And we're going to do all of that in    minutes  I promise. I want to start with this slide  because this slide sort of tells the s

[841, 'Alessandra Orofino 2014', 'Alessandra Orofino', 'female', 'It’s our city. Let’s fix it', 'Nov 2014', 908, "Fifty four percent of the world's population lives in our cities. In developing countries  one third of that population is living in slums. Seventy five percent of global energy consumption occurs in our cities  and    percent of gas emissions that cause global warming come from our cities. So things that you and I might think about as global problems  like climate change  the energy crisis or poverty  are really  in many ways  city problems. They will not be solved unless people who live in cities  like most of us  actually start doing a better job  because right now  we are  not doing a very good one. And that becomes very clear when we look into three aspects of city life  first  our citizens' willingness to engage with democratic institutions  second  our cities' ability to really include all of their residents  and lastly  our own ability to live fulfilling and happy l

[1181, 'Alex Kipman 2016', 'Alex Kipman', 'male', 'A futuristic vision of the age of holograms', 'Mar 2016', 1141, "Thousands of years from now  we'll look back at the first century of computing as a fascinating but very peculiar time    the only time in history where humans were reduced to live in  D space  interacting with technology as if we were machines  a singular      year period in the vastness of time where humans communicated  were entertained and managed their lives from behind a screen. Today  we spend most of our time tapping and looking at screens. What happened to interacting with each other  I don't know about you  but I feel limited inside this  D world of monitors and pixels. And it is this very limitation and my desire to connect with people that inspires me as a creator. Put simply  I want to create a new reality  a reality where technology brings us infinitely closer to each other  a reality where people  not devices  are the center of everything. I dream of a real

[1454, 'Alex Tabarrok 2009', 'Alex Tabarrok', 'male', 'How ideas trump crises', 'Apr 2009', 791, "The first half of the   th century was an absolute disaster in human affairs  a cataclysm. We had the First World War  the Great Depression  the Second World War and the rise of the communist nations. And each one of these forces split the world  tore the world apart  divided the world. And they threw up walls    political walls  trade walls  transportation walls  communication walls  iron curtains    which divided peoples and nations. It was only in the second half of the   th century that we slowly began to pull ourselves out of this abyss. Trade walls began to come tumbling down. Here are some data on tariffs  starting at    percent  coming down to less than   percent. We globalized the world. And what does that mean  It means that we extended cooperation across national boundaries  we made the world more cooperative. Transportation walls came tumbling down. You know in      the typical

[965, 'Alice Goffman 2015', 'Alice Goffman', 'female', "How we're priming some kids for college — and others for prison", 'May 2015', 952, "On the path that American children  travel to adulthood  two institutions oversee the journey. The first is the one we hear  a lot about  college. Some of you may remember the excitement that you felt when you first set off for college. Some of you may be in college right now and you're feeling this excitement at this very moment. College has some shortcomings. It's expensive  it leaves young people in debt. But all in all  it's a pretty good path. Young people emerge from college with pride and with great friends and with a lot of knowledge  about the world. And perhaps most importantly  a better chance in the labor market than they had before they got there. Today I want to talk about the second institution overseeing the journey from childhood to adulthood in the United States. And that institution is prison. Young people on this journey are mee

[878, 'Alison Jackson 2008', 'Alison Jackson', 'female', 'An unusual glimpse at celebrity', 'Jan 2008', 1049, "I'm a contemporary artist and I show in art galleries and museums. I show a number of photographs and films  but I also make television programs  books and some advertising  all with the same concept. And it's about our fixation with celebrity and celebrity culture  and the importance of the image  celebrity is born of photography. I'm going to start with how I started with this concept seven years ago  when Princess Diana died. There was a sort of a standstill in Britain the moment of her death  and people decided to mourn her death in a sort of mass way. I was fascinated by this phenomenon  so I wondered  could one erase the image of Diana  actually quite crudely and physically  So  I got a gun and started to shoot at the image of Diana  but I couldn't erase this from my memory and certainly it was not being erased from the public psyche. Momentum was being built. The press 

KeyboardInterrupt: 

In [31]:
UMtalks6

,Unnamed: 0,Unnamed: 0.1,citation,author,gender,title,date,length,text,occupation,numDate,ValenceScore
0,1826,1828,Aaron Huey 2010,Aaron Huey,male,America's native prisoners of war,Nov 2010,911,I'm here today to show my photographs of the L...,Photographer,201011,5.533945
1,1969,1971,Aaron Koblin 2011,Aaron Koblin,male,Visualizing ourselves ... with crowd-sourced data,May 2011,1083,So I think data can actually make us more huma...,Data artist,201105,6.150012
2,1249,1251,Adam Grosser 2008,Adam Grosser,male,A mobile fridge for vaccines,Jun 2008,201,This is a work in process based on some comme...,Venture capitalist,200806,5.921532
3,1422,1424,Adam Savage 2009,Adam Savage,male,My obsession with objects and the stories they...,Mar 2009,881,About four years ago the New Yorker published...,"Maker, critical thinker",200903,6.131734
4,1669,1671,Adora Svitak 2010,Adora Svitak,female,What adults can learn from kids,Apr 2010,479,Now I want to start with a question When was...,Child prodigy,201004,6.190227
5,1951,1953,Aicha el-Wafi + Phyllis Rodriguez 2011,Aicha el-Wafi + Phyllis Rodriguez,female,"The mothers who found forgiveness, friendship",May 2011,570,Phyllis Rodriguez We are here today because o...,9/11 mothers,201105,5.891186
6,1389,1391,Aimee Mullins 2009,Aimee Mullins,female,Changing my legs - and my mindset,Jan 2009,1222,Sheryl Shade Hi Aimee. Aimee Mullins Hi. SS...,Athlete and actor,200901,5.990806
7,1415,1417,Aimee Mullins 2009,Aimee Mullins,female,My 12 pairs of legs,Mar 2009,484,I was speaking to a group of about kids a...,Athlete and actor,200903,6.149972
8,1461,1463,Al Gore 2009,Al Gore,male,What comes after An Inconvenient Truth?,May 2009,445,Last year I showed these two slides so that de...,Climate advocate,200905,5.987343
9,1879,1881,Al Seckel 2007,Al Seckel,male,Visual illusions that show how we (mis)think,Apr 2007,778,We're going to talk my a new lecture ju...,Master of visual illusions,200704,6.260313


In [26]:
len(mnds)

1083

In [10]:
out = pd.read_csv('../data/talks_7.csv')
len(out)

1545

In [4]:
out

,slug,citation,author,gender,title,published,length,text,occupation,numPub,ValenceScore,view_count,event,filmed,numFilmed
0,aj_jacobs_how_healthy_living_nearly_killed_me,AJ Jacobs 2012,AJ Jacobs,male,How healthy living nearly killed me,Jan 2012,509,I've spent the last decade subjecting myself t...,Author,201201,6.099078,1511571,TEDMED 2011,2011-10-28,20111028
1,aj_jacobs_the_world_s_largest_family_reunion_w...,AJ Jacobs 2014,AJ Jacobs,male,The world's largest family reunion … we're all...,Jun 2014,577,Six months ago I got an email from a man in I...,Author,201406,6.267381,1123811,TEDActive 2014,2014-03-20,20140320
2,a_j_jacobs_year_of_living_biblically,AJ Jacobs 2008,AJ Jacobs,male,My year of living biblically,Jul 2008,1023,I thought I'd tell you a little about what I l...,Author,200807,6.075994,2419084,EG 2007,2007-12-02,20071202
3,aaron_o_connell_making_sense_of_a_visible_quan...,Aaron O'Connell 2011,Aaron O'Connell,male,Making sense of a visible quantum object,Jun 2011,459,This is a representation of your brain and yo...,Physicist,201106,6.058835,1290118,TED2011,2011-03-02,20110302
4,abe_davis_new_video_technology_that_reveals_an...,Abe Davis 2015,Abe Davis,male,New video technology that reveals an object's ...,May 2015,1066,Most of us think of motion as a very visual th...,Computer scientist,201505,6.269410,1405188,TED2015,2015-03-17,20150317
5,abha_dawesar_life_in_the_digital_now,Abha Dawesar 2013,Abha Dawesar,female,"Life in the ""digital now""",Oct 2013,708,I was in New York during Hurricane Sandy and ...,Novelist,201310,6.091329,1263264,TEDGlobal 2013,2013-06-14,20130614
6,abigail_washburn_building_us_china_relations_b...,Abigail Washburn 2012,Abigail Washburn,female,Building US-China relations ... by banjo,Apr 2012,373,If you had caught me straight out of college i...,Clawhammer banjo player,201204,6.322439,989838,TED2012,2012-02-27,20120227
7,abraham_verghese_a_doctor_s_touch,Abraham Verghese 2011,Abraham Verghese,male,A doctor's touch,Sep 2011,1090,A few months ago a year old woman came to ...,Physician and author,201109,5.877625,1432826,TEDGlobal 2011,2011-07-14,20110714
8,achenyo_idachaba_how_i_turned_a_deadly_plant_i...,Achenyo Idachaba 2016,Achenyo Idachaba,female,How I turned a deadly plant into a thriving bu...,Jan 2016,431,Welcome to Bayeku a riverine community in Iko...,Green entrepreneur,201601,6.207708,1743873,TEDWomen 2015,2015-05-27,20150527
9,adam_davidson_what_we_learned_from_teetering_o...,Adam Davidson 2012,Adam Davidson,male,What we learned from teetering on the fiscal c...,Dec 2012,1132,So a friend of mine who's a political scientis...,Journalist,201212,5.755019,787596,TEDSalon NY2012,2012-12-17,20121217


In [49]:
meta2 = pd.read_csv('../../tedmetadata/unmatched_meta2.csv')
talks2 = pd.read_csv('../data/unmatched_talks6e2.csv')

In [3]:
A = meta2.iloc[0]["slug"].lower().split('_')

In [4]:
B = talks2.iloc[1]["title"].lower().split(' ')

In [7]:
len(set(A) & set(B)) >0.8*len(B)

True

In [6]:
set(B)

{'forgiveness,', 'found', 'friendship', 'mothers', 'the', 'who'}

In [17]:
# Next wave of matching. We will match based on the title to make more rows in *talks_7.csv*

# We will follow nearly the same procedure as before EXCEPT that the unmatch files are already sorted. 

# Step 1 - Import the files & get the headers: 
read_talks = csv.reader(open('../data/talks_7.csv'))
headers = next(read_talks)

UMtalks6two = pd.read_csv('../data/unmatched_talks6e2.csv')
UMmeta2 = pd.read_csv('../../tedmetadata/unmatched_meta2.csv')

# Step 2 - DNE

# Step 3 - Create an index list for *unmatched_meta.csv*
mnds = list(range(len(UMmeta2)))

# Step 4 - Loop over the rows in *talks_6e.csv*
with open('../data/talks_7add2.csv', 'w') as csvfile:
    # Make the output file
    outfile = csv.writer(csvfile)
    outfile.writerow(headers)
    
    matchless_inds = []
    
    # Loop over each row in *talks_6e.csv*
    for inds in range(len(UMtalks6two)):
        talk_row = UMtalks6two.iloc[inds]
        aulist = talk_row["author"].lower().split()
        tlist = talk_row["title"].lower().split()
        
        # Set MATCH flag
        match = False
        good_inds = []
        
        for m in mnds:
            # Grab row associated to index M in *unmatched_meta.csv*
            row2 = UMmeta2.iloc[m]
            slist = row2['slug'].lower().split('_')
            
            # Compare author and slug lists:
            if len(set(slist) & set(tlist)) > 0.7*len(tlist):
                # If comparison goes well, add this index to GOOD_INDS
                good_inds.append(m)
                print("match!")
                print(good_inds)
                
        #print("For talks row", str(inds), "there are ", str(len(good_inds)), "matches")
        if len(good_inds) == 1:
            # if we find exactly one match by author, then we can add the
            # the row to the output. 
            gi = good_inds[0]
            row_gi = UMmeta2.iloc[gi]
            
            add_row = [row_gi['slug']] + talk_row.tolist()[1:]
            add_row.append(row_gi['view_count'])
            add_row.append(row_gi['event'])
            add_row.append(row_gi['filmed'])
            numFilmed = ''.join(row_gi["filmed"].split('-'))
            add_row.append(numFilmed)
            
            print(add_row)
            
            outfile.writerow(add_row)
                
            mnds.remove(good_inds[0])
        
        else :
            matchless_inds.append(inds)

unmatchedMeta3 = UMmeta2.iloc[mnds]
unmatchedMeta3.to_csv('../../tedmetadata/unmatched_meta3.csv')
unmatchedTalks3 = UMtalks6two.iloc[matchless_inds]
unmatchedTalks3.to_csv('../data/unmatched_talks6e3.csv')

match!
[0]
['9_11_healing_the_mothers_who_found_forgiveness_friendship', 1951, 1953, 'Aicha el-Wafi + Phyllis Rodriguez 2011', 'Aicha el-Wafi + Phyllis Rodriguez', 'female', 'The mothers who found forgiveness, friendship', 'May 2011', 570, "Phyllis Rodriguez  We are here today because of the fact that we have what most people consider an unusual friendship. And it is. And yet  it feels natural to us now. I first learned that my son had been in the World Trade Center on the morning of September   th      . We didn't know if he had perished yet until    hours later. At the time  we knew that it was political. We were afraid of what our country was going to do in the name of our son    my husband  Orlando  and I and our family. And when I saw it    and yet  through the shock  the terrible shock  and the terrible explosion in our lives  literally  we were not vengeful. And a couple of weeks later when Zacarias Moussaoui was indicted on six counts of conspiracy to commit terrorism  and the 

match!
[118]
['christopher_m00t_poole_the_case_for_anonymity_online', 1715, 1717, 'Christopher "moot" Poole" 2010', 'Christopher "moot" Poole"', 'male', 'The case for anonymity online', 'Jun 2010', 670, "Tom Green  That's a  chan thing. These kids on the Internet  they have this group of kids and they like to say funny words like  barrel roll.  It's a video game move from  Star Fox.   Star Fox       Assistant   Star Fox   .   Tom Green  Yeah. And they've been dogging me for a year. I got to tell you  it's driving me nuts  actually. Sometimes I wake up in the middle of the night and I scream    chan   Christopher Poole  When I was     I found this website called Futaba Channel. And it was a Japanese forum and imageboard. That format of forum  at that time  was not well known outside of Japan. And so what I did is I took it  I translated it into English  and I stuck it up for my friends to use. Now  six and a half years later  over seven million people are using it  contributing over    

match!
[165]
['defend_our_freedom_to_share_or_why_sopa_is_a_bad_idea', 60, 62, 'Clay Shirky 2012', 'Clay Shirky', 'male', 'Why SOPA is a bad idea', 'Jan 2012', 823, "I'm going to start here. This is a hand lettered sign that appeared in a mom and pop bakery in my old neighborhood in Brooklyn a few years ago. The store owned one of those machines that can print on plates of sugar. And kids could bring in drawings and have the store print a sugar plate for the top of their birthday cake. But unfortunately  one of the things kids liked to draw was cartoon characters. They liked to draw the Little Mermaid  they'd like to draw a smurf  they'd like to draw Micky Mouse. But it turns out to be illegal to print a child's drawing of Micky Mouse onto a plate of sugar. And it's a copyright violation. And policing copyright violations for children's birthday cakes was such a hassle that the College Bakery said   You know what  we're getting out of that business. If you're an amateur  you don't have

match!
[660]
['the_interspecies_internet_an_idea_in_progress', 493, 495, 'Diana Reiss, Peter Gabriel, Neil Gershenfeld and Vint Cerf 2013', 'Diana Reiss, Peter Gabriel, Neil Gershenfeld and Vint Cerf', 'both', 'The interspecies internet? An idea in progress', 'Jul 2013', 11, "Diana Reiss  You may think you're looking through a window at a dolphin spinning playfully  but what you're actually looking through is a two way mirror at a dolphin looking at itself spinning playfully. This is a dolphin that is self aware. This dolphin has self awareness. It's a young dolphin named Bayley. I've been very interested in understanding the nature of the intelligence of dolphins for the past    years. How do we explore intelligence in this animal that's so different from us  And what I've used is a very simple research tool  a mirror  and we've gained great information  reflections of these animal minds. Dolphins aren't the only animals  the only non human animals  to show mirror self recognition. We

match!
[4]
['a_whistleblower_you_haven_t_heard', 1891, 1893, 'Geert Chatrou 2011', 'Geert Chatrou', 'male', "A whistleblower you haven't heard", 'Feb 2011', 708, "Thank you. Thank you very much. That was whistling. I'm trying to do this in English. What is a chubby  curly haired guy from Holland    why is he whistling  Well actually  I've  been  whistling since the age of four  about four. My dad was always whistling around the house  and I just thought that's part of communication in my family. So I whistled along with him. And actually  till I was     I always annoyed and irritated people with whistling  because  to be honest  my whistling is a kind of deviant behavior. I whistled alone. I whistled in the classroom. I whistled on  my  bike. I whistled everywhere. And I also whistled at a Christmas Eve party with my family in law. And they had some  in my opinion  terrible Christmas music. And when I hear music that I don't like  I try to make it better. So  Rudolph the Red Nosed Rein

...

match!
[661]
['the_jill_and_julia_show', 986, 988, 'Jill Sobule + Julia Sweeney 2008', 'Jill Sobule + Julia Sweeney', 'female', 'The Jill and Julia Show', 'Feb 2008', 363, "  Jill Sobule  At a conference in Monterey by the big  big jellyfish tank      I first saw you and I got so shy.     You see  I was a little paranoid 'cause I might have been high.     And I hadn't done that in ages and I won't do that again.     But that's another story.     Loved you forever and I've been a big fan      the one woman shows  I even rented  Pat.      I got enough nerve to come up to you      but little did I know one year later we'd be doing this show.     I sing. Julia Sweeney  I tell stories. Together  The Jill and Julia Show.     Sobule  Sometimes it works. Sweeney  Sometimes it doesn't.     Together  The Jill and Julia Show.     Sweeney  At a conference in Monterey next to the big  big jellyfish tank      I first saw you and I wasn't so shy.   I made a beeline for you and told you what a huge fa

match!
[693]
['two_young_scientists_break_down_plastics_with_bacteria', 499, 501, 'Miranda Wang and Jeanny Yao 2013', 'Miranda Wang and Jeanny Yao', 'they (both female)', 'Two young scientists break down plastics with bacteria', 'Jul 2013', 11, "Miranda Wang  We're here to talk about accidents. How do you feel about accidents  When we think about accidents  we usually consider them to be harmful  unfortunate or even dangerous  and they certainly can be. But are they always that bad  The discovery that had led to penicillin  for example  is one of the most fortunate accidents of all time. Without biologist Alexander Fleming's moldy accident  caused by a neglected workstation  we wouldn't be able to fight off so many bacterial infections. Jeanny Yao  Miranda and I are here today because we'd like to share how our accidents have led to discoveries. In       we visited the Vancouver Waste Transfer Station and saw an enormous pit of plastic waste. We realized that when plastics get to the d

match!
[482]
match!
[482, 483]
match!
[483]
['nicholas_negroponte_on_one_laptop_per_child_two_years_on', 1253, 1255, 'Nicholas Negroponte 2008', 'Nicholas Negroponte', 'male', 'One Laptop per Child, two years on', 'Jun 2008', 984, "Most people don't know that when I went to high school in this country    I applied for university at a time when I was convinced I was going to be an artist and be a sculptor. And I came from a very privileged background. I was very lucky. My family was wealthy  and my father believed in one thing  and that was to give us all as much education as we wanted. And I announced I wanted to be a sculptor in Paris. And he was a clever man. He sort of said   Well  that's OK  but you've done very well in your math SATs.  In fact  I'd got an    . And he thought I did very well    and I did  too    in the arts  this was my passion. And he said  If you go to MIT   to which I had been given early admission   I will pay for every year you're at MIT  in graduate or underg

match!
[160]
['debate_does_the_world_need_nuclear_energy', 1721, 1723, 'Stewart Brand + Mark Z. Jacobson 2010', 'Stewart Brand + Mark Z. Jacobson', 'male', 'Debate: Does the world need nuclear energy?', 'Jun 2010', 1365, "Chris Anderson  We're having a debate. The debate is over the proposition   What the world needs now is nuclear energy.  True or false  And before we have the debate  I'd like to actually take a show of hands    on balance  right now  are you for or against this  So those who are  yes   raise your hand.  For.  Okay  hands down. Those who are against  raise your hands. Okay  I'm reading that at about    to    in favor at the start. Which means we're going to take a vote at the end and see how that shifts  if at all. So here's the format  They're going to have six minutes each  and then after one little  quick exchange between them  I want two people on each side of this debate in the audience to have    seconds to make one short  crisp  pungent  powerful point. So  in 

match!
[682]
['tom_shannon_the_painter_and_the_pendulum', 1628, 1630, 'Tom Shannon, John Hockenberry 2010', 'Tom Shannon, John Hockenberry', 'male', 'The painter and the pendulum', 'Feb 2010', 788, "John Hockenberry  It's great to be here with you  Tom. And I want to start with a question that has just been consuming me since I first became familiar with your work. In you work there's always this kind of hybrid quality of a natural force in some sort of interplay with creative force. Are they ever in equilibrium in the way that you see your work  Tom Shannon  Yeah  the subject matter that I'm looking for  it's usually to solve a question. I had the question popped into my head  What does the cone that connects the sun and the Earth look like if you could connect the two spheres  And in proportion  what would the size of the sphere and the length  and what would the taper be to the Earth  And so I went about and made that sculpture  turning it out of solid bronze. And I did one that was

In [12]:
len(mnds)

704

In [51]:
meta3 = pd.read_csv('../../tedmetadata/unmatched_meta3.csv')
talks3 = pd.read_csv('../data/unmatched_talks6e3.csv')

In [52]:
meta3

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,slug,view_count,event,filmed
0,1,1,1,a_choir_as_big_as_the_internet,400751,Eric Whitacre's Virtual Choir,2010-03-10
1,3,3,4,a_ted_speaker_s_worst_nightmare,2984064,TED2012,2012-03-01
2,5,5,6,aakash_odedra_a_dance_in_a_hurricane_of_paper_...,831464,TEDGlobal 2014,2014-10-21
3,6,6,7,aala_el_khani_what_it_s_like_to_be_a_parent_in...,943318,TEDxManchester,2016-02-14
4,7,9,13,abigail_marsh_why_some_people_are_more_altruis...,1775619,TEDSummit,2016-06-26
5,8,10,17,adam_alter_why_our_screens_make_us_less_happy,2613880,TED2017,2017-04-24
6,9,11,19,adam_de_la_zerda_how_we_can_start_winning_the_...,1010021,TEDxStanford,2016-04-24
7,10,12,20,adam_driver_my_journey_from_marine_to_actor,4588988,TED Talks Live,2015-11-03
8,11,13,22,adam_galinsky_how_to_speak_up_for_yourself,3430090,TEDxNewYork,2016-09-10
9,12,14,24,adam_grant_are_you_a_giver_or_a_taker,3917466,TED@IBM,2016-11-15


In [53]:
talks3

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1,citation,author,gender,title,date,length,text,occupation,numDate,ValenceScore
0,0,3,1424,1424,Adam Savage 2009,Adam Savage,male,My obsession with objects and the stories they...,Mar 2009,881,About four years ago the New Yorker published...,"Maker, critical thinker",200903,6.131734
1,2,6,1391,1391,Aimee Mullins 2009,Aimee Mullins,female,Changing my legs - and my mindset,Jan 2009,1222,Sheryl Shade Hi Aimee. Aimee Mullins Hi. SS...,Athlete and actor,200901,5.990806
2,3,7,1417,1417,Aimee Mullins 2009,Aimee Mullins,female,My 12 pairs of legs,Mar 2009,484,I was speaking to a group of about kids a...,Athlete and actor,200903,6.149972
3,4,8,1463,1463,Al Gore 2009,Al Gore,male,What comes after An Inconvenient Truth?,May 2009,445,Last year I showed these two slides so that de...,Climate advocate,200905,5.987343
4,5,16,1847,1847,Alex Steffen 2007,Alex Steffen,male,The route to a sustainable future,Apr 2007,1039,When I'm starting talks like this I usually d...,Planetary futurist,200704,6.239599
5,6,17,2025,2025,Alex Steffen 2011,Alex Steffen,male,The shareable future of cities,Aug 2011,600,Climate change is already a heavy topic and i...,Planetary futurist,201108,6.264768
6,7,28,1069,1069,Anders Fjellberg 2015,Anders Fjellberg,male,Two nameless bodies washed up on the beach. He...,Oct 2015,874,So this right here is the tiny village of Elle...,Journalist,201510,5.909177
7,8,31,1139,1139,Andrés Ruzo 2016,Andrés Ruzo,male,How I found a mythical boiling river in the Am...,Feb 2016,939,As a boy in Lima my grandfather told me a leg...,NaN,201602,6.264641
8,9,38,712,712,Arthur Benjamin 2007,Arthur Benjamin,male,"A performance of ""Mathemagic""",Dec 2007,904,Good morning ladies and gentlemen. My name is...,Mathemagician,200712,6.124810
9,10,39,1499,1499,Arthur Benjamin 2009,Arthur Benjamin,male,Teach statistics before calculus!,Jun 2009,153,Now if President Obama invited me to be the n...,Mathemagician,200906,6.112917


In [22]:
talks7 = pd.read_csv('../data/talks_7.csv')
talks_Add1 = pd.read_csv('../data/talks_7add1.csv')
talks_Add2 = pd.read_csv('../data/talks_7add2.csv')

In [23]:
talks_Add1

,slug,citation,author,gender,title,published,length,text,occupation,numPub,ValenceScore,view_count,event,filmed,numFilmed
aaron_huey,1828,Aaron Huey 2010,Aaron Huey,male,America's native prisoners of war,Nov 2010,911,I'm here today to show my photographs of the L...,Photographer,201011,5.533945,1404833,TEDxDU 2010,2010-09-19,20100919
aaron_koblin,1971,Aaron Koblin 2011,Aaron Koblin,male,Visualizing ourselves ... with crowd-sourced data,May 2011,1083,So I think data can actually make us more huma...,Data artist,201105,6.150012,1503044,TED2011,2011-03-02,20110302
adam_grosser_and_his_sustainable_fridge,1251,Adam Grosser 2008,Adam Grosser,male,A mobile fridge for vaccines,Jun 2008,201,This is a work in process based on some comme...,Venture capitalist,200806,5.921532,867800,TED2007,2007-02-02,20070202
adora_svitak,1671,Adora Svitak 2010,Adora Svitak,female,What adults can learn from kids,Apr 2010,479,Now I want to start with a question When was...,Child prodigy,201004,6.190227,4899371,TED2010,2010-02-13,20100213
al_seckel_says_our_brains_are_mis_wired,1881,Al Seckel 2007,Al Seckel,male,Visual illusions that show how we (mis)think,Apr 2007,778,We're going to talk my a new lecture ju...,Master of visual illusions,200704,6.260313,2179841,TED2004,2004-02-26,20040226
alain_de_botton_atheism_2_0,60,Alain de Botton 2012,Alain de Botton,male,Atheism 2.0,Jan 2012,1148,One of the most common ways of dividing the wo...,Philosopher,201201,6.047988,2264873,TEDGlobal 2011,2011-07-13,20110713
alan_russell_on_regenerating_our_bodies,197,Alan Russell 2007,Alan Russell,male,The potential of regenerative medicine,Jul 2007,1110,I'm going to talk to you today about hopefully...,Medical futurist,200707,5.860705,1468307,TED2006,2006-02-02,20060202
alan_siegel_let_s_simplify_legal_jargon,1664,Alan Siegel 2010,Alan Siegel,male,Let's simplify legal jargon!,Mar 2010,251,So basically we have public leaders public o...,"Executive, simplicity guru",201003,6.112521,895061,TED2010,2010-02-11,20100211
alessandra_orofino_it_s_our_city_let_s_fix_it,841,Alessandra Orofino 2014,Alessandra Orofino,female,It’s our city. Let’s fix it,Nov 2014,908,Fifty four percent of the world's population l...,Political mobilization activist,201411,6.105815,755907,TEDGlobal 2014,2014-10-07,20141007
alessandro_acquisti_why_privacy_matters,562,Alessandro Acquisti 2013,Alessandro Acquisti,male,What will a future without secrets look like?,Oct 2013,888,I would like to tell you a story connecting th...,Privacy economist,201310,6.166146,1328558,TEDGlobal 2013,2013-06-14,20130614


In [15]:
talks7

,slug,citation,author,gender,title,published,length,text,occupation,numPub,ValenceScore,view_count,event,filmed,numFilmed
0,aj_jacobs_how_healthy_living_nearly_killed_me,AJ Jacobs 2012,AJ Jacobs,male,How healthy living nearly killed me,Jan 2012,509,I've spent the last decade subjecting myself t...,Author,201201,6.099078,1511571,TEDMED 2011,2011-10-28,20111028
1,aj_jacobs_the_world_s_largest_family_reunion_w...,AJ Jacobs 2014,AJ Jacobs,male,The world's largest family reunion … we're all...,Jun 2014,577,Six months ago I got an email from a man in I...,Author,201406,6.267381,1123811,TEDActive 2014,2014-03-20,20140320
2,a_j_jacobs_year_of_living_biblically,AJ Jacobs 2008,AJ Jacobs,male,My year of living biblically,Jul 2008,1023,I thought I'd tell you a little about what I l...,Author,200807,6.075994,2419084,EG 2007,2007-12-02,20071202
3,aaron_o_connell_making_sense_of_a_visible_quan...,Aaron O'Connell 2011,Aaron O'Connell,male,Making sense of a visible quantum object,Jun 2011,459,This is a representation of your brain and yo...,Physicist,201106,6.058835,1290118,TED2011,2011-03-02,20110302
4,abe_davis_new_video_technology_that_reveals_an...,Abe Davis 2015,Abe Davis,male,New video technology that reveals an object's ...,May 2015,1066,Most of us think of motion as a very visual th...,Computer scientist,201505,6.269410,1405188,TED2015,2015-03-17,20150317
5,abha_dawesar_life_in_the_digital_now,Abha Dawesar 2013,Abha Dawesar,female,"Life in the ""digital now""",Oct 2013,708,I was in New York during Hurricane Sandy and ...,Novelist,201310,6.091329,1263264,TEDGlobal 2013,2013-06-14,20130614
6,abigail_washburn_building_us_china_relations_b...,Abigail Washburn 2012,Abigail Washburn,female,Building US-China relations ... by banjo,Apr 2012,373,If you had caught me straight out of college i...,Clawhammer banjo player,201204,6.322439,989838,TED2012,2012-02-27,20120227
7,abraham_verghese_a_doctor_s_touch,Abraham Verghese 2011,Abraham Verghese,male,A doctor's touch,Sep 2011,1090,A few months ago a year old woman came to ...,Physician and author,201109,5.877625,1432826,TEDGlobal 2011,2011-07-14,20110714
8,achenyo_idachaba_how_i_turned_a_deadly_plant_i...,Achenyo Idachaba 2016,Achenyo Idachaba,female,How I turned a deadly plant into a thriving bu...,Jan 2016,431,Welcome to Bayeku a riverine community in Iko...,Green entrepreneur,201601,6.207708,1743873,TEDWomen 2015,2015-05-27,20150527
9,adam_davidson_what_we_learned_from_teetering_o...,Adam Davidson 2012,Adam Davidson,male,What we learned from teetering on the fiscal c...,Dec 2012,1132,So a friend of mine who's a political scientis...,Journalist,201212,5.755019,787596,TEDSalon NY2012,2012-12-17,20121217
